### **Software info**

|Software     |Version|
|-------------|-------|
|`python`|3.13.0|
|`ipykernel`|[6.29.5](https://pypi.org/project/ipykernel/)|
|`pandas`|[2.2.3](https://pypi.org/project/pandas/)|
|`Biopython`|[1.84](https://pypi.org/project/biopython/)|
|`mafft`|[7.525](https://anaconda.org/bioconda/mafft)|
|`trimAl`|[1.5.0](https://anaconda.org/bioconda/trimal)|
|`iqtree2`|[2.3.6](https://anaconda.org/bioconda/iqtree)|

Conda envinronment: `phoacr.yaml`<br>
Install the envinronment with:

In [ ]:
! conda env create -f ../phoacr.yaml

Reload `VS Code` (close & open), then activate this envinronment as kernel

### **Hardware info**

- OS: Ubuntu 22.04 (Windows Subsystem for Linux)
- CPU: Intel Xeon E5-2670v3
- RAM: 32GB (16GB for WSL)

In [1]:
! lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  24
  On-line CPU(s) list:   0-23
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU E5-2670 v3 @ 2.30GHz
    CPU family:          6
    Model:               63
    Thread(s) per core:  2
    Core(s) per socket:  12
    Socket(s):           1
    Stepping:            2
    BogoMIPS:            4589.37
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                         logy cpuid pni pclmulqdq vmx ssse3 fma cx16 pcid sse4_1
                          sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervis
                         or lahf_lm abm invpcid_single pti ssbd i

### **Step 0. Install `phyloki`**

Download `phyloki` from `GitHub` repo

In [ ]:
! wget https://raw.githubusercontent.com/iliapopov17/phyloki/refs/heads/v0.525/phyloki.py

Import `phyloki`

In [2]:
from phyloki import *

### **Step 1. Download sequences**

`data/accession_numbers.txt` file was created manually

In [ ]:
get_sequences('iljapopov17@gmail.com', 'data/accession_numbers.txt', 'genbank_sequences')

Check the number of downloaded sequences

In [4]:
! ls genbank_sequences/| wc -l

64


Combine them to one file

In [5]:
! cat genbank_sequences/*.fasta > data/all_seqs.fa

Check the number of sequences in combined file

In [6]:
with open("data/all_seqs.fa", "r") as fasta_file:
    content = fasta_file.read()
    num_sequences = content.count(">")
print(f"The number of sequences in combined file: {num_sequences}")

The number of sequences in combined file: 64


### **Step 2. Multiple Sequence Alignment**

In [ ]:
! mafft --auto data/all_seqs.fa > data/all_seqs_mafft.fa

### **Step 3. MSA trimming**

In [ ]:
! trimal -in data/all_seqs_mafft.fa -out data/all_seqs_mafft_trim.fa -automated1

### **Step 4. Launching `ModelFinder` to get the best substitution model**

Make directory to store log files of `ModelFinder`

In [10]:
! mkdir model-finder/

Run `ModelFinder`

In [ ]:
! iqtree2 -m MFP -s data/all_seqs_mafft_trim.fa --prefix model-finder/tree_MF2 -T AUTO -redo

Get the best substitution model

In [12]:
! head -42 model-finder/tree_MF2.iqtree | tail -6

Best-fit model according to BIC: GTR+F+I+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
GTR+F+I+G4        -7124.403   14514.806 +    0.991   14663.944 +        1   15036.376 +        1


### **Step 5. Building the final tree**

Make directory to store files of `IQTREE`

In [13]:
! mkdir tree/

Run `IQTREE`

In [ ]:
! iqtree2 -s data/all_seqs_mafft_trim.fa -m GTR+F+I+G4 -pre tree/tree_ufb -bb 1000 -nt AUTO

### **Step 7. Fetch metadata for further tree annotation**

Make directory to store `metadata`

In [15]:
! mkdir metadata/

Run `phyloki` function `fetch_metadata()`

In [17]:
fetch_metadata('iljapopov17@gmail.com', 'data/accession_numbers.txt', 'metadata/raw_metadata.tsv')

The request has been fulfilled.
File saved to metadata/raw_metadata.tsv


Import `pandas`

In [18]:
import pandas as pd

Function to clean the `Year` column

In [19]:
def clean_year(input_file, output_file):
    """
    Clean the 'Year' column in a metadata .tsv file to extract only the last 4 digits.

    Args:
        input_file (str): Path to the input .tsv file.
        output_file (str): Path to save the cleaned .tsv file.
    """
    # Load the .tsv file into a DataFrame
    df = pd.read_csv(input_file, sep="\t")

    # Extract the last 4 digits of the 'Year' column
    df['Year'] = df['Year'].apply(lambda x: str(x)[-4:] if pd.notnull(x) else 'ND')

    # Save the updated DataFrame to a new .tsv file
    df.to_csv(output_file, sep="\t", index=False)

    print(f"The 'Year' column has been cleaned.\nFile saved to {output_file}")

Run `clean_year()` function

In [20]:
clean_year('metadata/raw_metadata.tsv', 'metadata/metadata.tsv')

The 'Year' column has been cleaned.
File saved to metadata/metadata.tsv


### **Step 8. Tree visualization**

Please open `RStudio` and proceed to the `ggtree_journal.R` script

### **Step 9. Download a map**

It will be needed in the 3rd stage of the study `03_Map`

Download [`Natural Earth II with Shaded Relief, Water, and Drainages`](https://www.naturalearthdata.com/downloads/10m-natural-earth-2/10m-natural-earth-2-with-shaded-relief-water-and-drainages/) map

In [ ]:
! wget https://naturalearth.s3.amazonaws.com/10m_raster/NE2_HR_LC_SR_W_DR.zip

Unzip downloaded archive to the `../03_Map/NE2_HR_LC_SR_W_DR` directory

>Make sure `unzip` is installed in your system
>For `Ubuntu` please write in your terminal:

>```sudo apt-get install unzip```

In [ ]:
! unzip NE2_HR_LC_SR_W_DR.zip -d ../03_Map/NE2_HR_LC_SR_W_DR && rm -rf NE2_HR_LC_SR_W_DR.zip